In [2]:
import pandas as pd
import numpy as np
import math
# from bridge_specs import box
from Cross import calc_centroid as cg, calc_moi as moi
from irc6_2007 import *

In [3]:
#inputs

df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])
display(df)
www = df.loc[[('MaxSF','ClassA')]]
display(www.max())

0.00      6.25      12.50      18.75      25.00  \
MaxBM  ClassA       0.000  2361.037  4068.075   5120.737   5526.150   
       Class70RW    0.000  4826.025  8200.950  10182.475  10877.400   
       Class70RT    0.000  3651.438  6259.675   7823.212   8342.050   
MaxSF+ ClassA     445.318   376.622   307.926    238.122    169.426   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   
MaxSF- ClassA       0.000   -12.030   -51.234   -106.602   -175.162   
       Class70RW    0.000   -17.024   -87.536   -208.476   -332.476   
       Class70RT    0.000   -54.810  -143.010   -229.810   -316.610   
MaxSF  ClassA     445.318   376.622   307.926    238.122    175.162   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   

                      31.25     37.50     43.75    50.00  
MaxBM  ClassA      5340.038  4327.675  2494.638    0.000  
       Class70RW  10163.225  8053.750  4611.975    0.000  
       Class70RT   7823.588  6260.425  3652.563    0.000  
MaxSF+ ClassA        99.676    43.520    13.056    0.000  
       Class70RW    271.524   148.964    44.948    0.000  
       Class70RT    229.390   142.590    54.390    0.000  
MaxSF- ClassA      -244.966  -313.662  -383.466 -452.162  
       Class70RW   -458.476  -582.476  -708.476 -832.476  
       Class70RT   -404.810  -491.610  -579.810 -666.610  
MaxSF  ClassA       244.966   313.662   383.466  452.162  
       Class70RW    458.476   582.476   708.476  832.476  
       Class70RT    404.810   491.610   579.810  666.610

0.00     445.318
6.25     376.622
12.50    307.926
18.75    238.122
25.00    175.162
31.25    244.966
37.50    313.662
43.75    383.466
50.00    452.162
dtype: float64

In [5]:
def loader(unit_wt, length, width, height, type):
    if type == 'r':
        load = unit_wt*length*width*height
    elif type == 't':
        load = unit_wt*length*width*height*0.5
    
    return load
def z(base, clr, dim, x, type):
    """base = width of base along with taper
       clr = overhang part from where centroid is calculated
        dim = width of object whose lever arm is required
        type = shape of object
    """
    y = []
    if type == 'r':
        y.append(cg('rectangle',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't1':
        y.append(cg('triangle_2',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't2':
        y.append(cg('triangle_1',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])

def na(x,s,cf,dm): 
    mor = (s*cf)/1000
    while dm>=mor:
        x = int(input("Enter a new trial depth:"))
        cs = d-0.416*xu
        cf = (0.446*fck*beta1*(1000*abut['width_of_abut'])*xu)/1000 
        mor = (s*cf)/1000
    return x

Abutment details

In [38]:
#inputs
df=pd.read_excel('data/abutment.xlsx',index_col=None,header=None)
df2=pd.read_excel('data/abutment2.xlsx',index_col=None,header=None)
# print(df2.iloc[3,0])
bearing_ht = 0.3 #assumed including pedestal
abut = {
    'total_ht' :df.iloc[1,6]+df.iloc[0,2]+df.iloc[1,8],
    'wc' : 0.075,
    'backw_ht' :df.iloc[0,2], #including wearing cource
    'cap':df.iloc[1,4],
    'stem_ht': df.iloc[1,6],
    'bw_width':df.iloc[1,2],
    'edge_to_bearing_c':df.iloc[0,9],
    'bw_to_bearing_c': df.iloc[1,9],
    'cap_width':df.iloc[0,4],
    'stem_width':df.iloc[0,6],
    'be_taperwidth':df.iloc[0,7],
    'width_of_abut':df2.iloc[1,0],
    'ttl_stembase_width':df2.iloc[0,0],
    'approach_slab_dep':df.iloc[1,8],
    'approach_slab_len':df.iloc[0,8],
    'haunch':{'stem_haunch':[df.iloc[0,5],df.iloc[1,5]],'approach_slab_haunch_rec':[df.iloc[0,0],df.iloc[1,0]], 'approach_slab_haunch_tri':[df.iloc[0,1],df.iloc[1,1]]} #list of width and height.
}
fdn = {
    'heel' : df2.iloc[3,0],
    'toe' : df2.iloc[2,0],
    'th' : df2.iloc[4,0],
    'wid' : df2.iloc[5,0],
    'len' : df2.iloc[2,0]+df2.iloc[3,0]+df2.iloc[0,0]
}
# print(abut['stem_ht'])

Inputs: Live Loads, Dead loads and Pedestrian loads

In [29]:
#dummy loads
df1=pd.read_csv('outputs/DL_for_Seismic.csv',index_col=None,header=None)
# print(df1)
dl = df1.iloc[1,3]+df1.iloc[2,3] #dead loads
ll = 1000 #live loads including impact
pdl = df1.iloc[4,3] #pedestrian loads
sdl=df1.iloc[3,3] #Surface Loads

Input: Material properties

In [43]:
#Inputs


df1=pd.read_excel('data/material.xlsx',index_col=None,header=None)
# print(df1)
E = 200000 #Elasticity of steel
fck = df1.iloc[1,0] #grade of concrete = M30
fy = df1.iloc[2,0]#grade of steel = Fe500
unitwt_conc = 25 #unit weight of concrete = 25KN/m^3
unitwt_bfill = 20 #unit weight of backfill = 20KN/m^3

     0
0    0
1   35
2  500


Self weight loads of abutment

In [7]:
# comp = ['stem_rect','stem_tri','cap','backwall','stem_haunch','approach_slab_haunch']
loads = [loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['stem_width'],'r'),
         loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['be_taperwidth'],'t'),
         loader(unitwt_conc,abut['cap'],abut['width_of_abut'],abut['cap_width'],'r'),
         loader(unitwt_conc,abut['backw_ht'],abut['width_of_abut'],abut['bw_width'],'r'),
         loader(unitwt_conc,abut['haunch']['stem_haunch'][0],abut['width_of_abut'],abut['haunch']['stem_haunch'][1],'t'),
         loader(unitwt_conc,abut['haunch']['approach_slab_haunch_rec'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_rec'][1],'r'),
          loader(unitwt_conc,abut['haunch']['approach_slab_haunch_tri'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_tri'][1],'t')]
lever_arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['stem_width'],abut['haunch']['stem_haunch'][0],'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['be_taperwidth'],abut['haunch']['stem_haunch'][0]+abut['stem_width'],'t1'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['cap_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['bw_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][0],0,'t2'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_rec'][0],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_tri'][0],0,'t2'),5)]
moments = []
for index,i in enumerate(loads):
    moments.append(loads[index]*lever_arm[index])
A1 = pd.DataFrame({"Load":loads,"Lever arm":lever_arm,"Moment":moments},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
A1.loc[len(A1.index)] = [A1['Load'].sum(),None,A1['Moment'].sum()]
display(A1.rename(index={7:'Total'}))
display(A1.loc[7,'Load'])

,Load,Lever arm,Moment
Stem(rectangle),767.8125,-0.05000,-38.390625
Stem(triangle),30.7125,0.60833,18.683335
Cap,157.5000,-0.12500,-19.687500
Backwall,225.2250,-0.72500,-163.288125
Stem haunch,5.2500,-0.74167,-3.893768
Approach slab haunch(rec),13.1250,-1.00000,-13.125000
Approach slab haunch(tri),6.5625,-1.04167,-6.835959
Total,1206.1875,NaN,-226.537642


1206.1875

Loads from superstructure

In [8]:
load = [dl,ll+pdl,sdl]
arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5)]
moment = []
for index,i in enumerate(load):
    moment.append(load[index]*arm[index])
A2 = pd.DataFrame({'Load':load, 'Lever arm':arm,'Moment':moment}, index = ['Dead load','Live load','Surface load'])
display(A2)
# display(A2.iloc[1,0])

,Load,Lever arm,Moment
Dead load,3000,0.025,75.00
Live load,1200,0.025,30.00
Surface load,250,0.025,6.25


Earth Pressure as per clause 214 of IRC:6

In [9]:
#Inputs

phi = 33 #Angle of internal friction of soil
alpha = 0 #Angle which earth face of the wal makes with the vertical of the wall
beta = 0 #Slope of earth fill
delta = 2/3*phi if 2/3*phi<22.5 else 22.5 #Angle of friction between earth and earth fill
h = abut['total_ht']-abut['approach_slab_dep'] #Height of earthfill
print(delta)

22.0


In [10]:
ep = unitwt_bfill*h*ka(alpha, beta, delta, phi) #Earth pressure in KN/m^2.
print(ep)
force = 0.5*ep*abut['width_of_abut']*h #Force due to earth pressure in KN.
forc = [force*math.cos(math.radians(delta)),force*math.cos(math.radians(delta)),force*math.sin(math.radians(delta))] #Horizontal and vertical components.
larm = [0.42*h, 0.33*h, -(abut['stem_width']+abut['be_taperwidth'])/2 ] #0.42 times height of wall above base for dry soil and 0.33 for wet.
momen = []
for index,i in enumerate(forc):
    momen.append(forc[index]*larm[index])
A3 = pd.DataFrame({'Force(KN)':forc, 'Lever arm(m)':larm,'Moment(KNm)':momen}, index = ['Earth pressure Hz(dry)','Earth pressure Hz(wet)','Earth pressure Ver'])
display(A3)

35.437639999999995


,Force(KN),Lever arm(m),Moment(KNm)
Earth pressure Hz(dry),924.601823,2.814,2601.829531
Earth pressure Hz(wet),924.601823,2.211,2044.294631
Earth pressure Ver,373.563385,-0.675,-252.155285


Braking Forces

In [11]:
#Inputs

wt_70r = 1000 #Weight of 70R vehicle in KN.
classa_wt = 554 #Weight of Class A in KN.
pt = 1.2 #Point of application of load above the deck in m.
factor = 0.2 #Lane fact as per clause 211.2 of IRC 6

In [12]:
F = [1*wt_70r*factor, 2*classa_wt*factor]
la = pt+abut['total_ht']
m = max(F)*la
A4 = pd.DataFrame({'Force(KN)':max(F), 'Lever arm(m)':la,'Moment(KNm)':m}, index = ['Braking load'])
display(A4)

,Force(KN),Lever arm(m),Moment(KNm)
Braking load,221.6,8.2,1817.12


Temperature Forces

In [13]:
temp_var = 30 #Temperature variation in degree celsius
alp = 0.000012 #Coefficient of thermal expansion in m/deg. c/m
l = box['span'] #span of bridge
shr_str = 0.0002 #Strain due to shrinkage.
th_el = temp_var*alp*l #Thermal elongation in m.
shr_el = l*shr_str #Shrinkage elongation in m.
tot = (th_el+shr_el)/2 #Total elongation in m.
shr_rt = 1000 #Shear rating of elastomer bearing in KN/m/m^2
arb = 0.235644 #Area of bearing in m^2.
htb = 0.0340 #Height of bearing in m.
no = 3 #Number of bearings.
f = 1.1*tot*(shr_rt*arb)/htb
# print(box['span'])

In [14]:
frc = 3*f
La = abut['stem_ht']+abut['cap']+bearing_ht
A5 = pd.DataFrame({'Force(KN)':frc, 'Lever arm(m)':La,'Moment(KNm)': (frc*La)}, index = ['Temperature force'])
display(A5)

,Force(KN),Lever arm(m),Moment(KNm)
Temperature force,320.198612,3.725,1192.739829


Live Load Surcharge as per clause 214.1.1.3 of IRC 6

In [15]:
hl = 1.2 
Pr = ka(alpha, beta, delta, phi)*hl*unitwt_bfill #Horizontal pressure 
print(Pr)
A6 = pd.DataFrame({'Force(KN)':(Pr*abut['width_of_abut']*(abut['total_ht']-abut['approach_slab_dep'])), 'Lever arm(m)':abut['total_ht']/3,'Moment(KNm)':((Pr*abut['width_of_abut']*(abut['total_ht']-abut['approach_slab_dep']))*abut['total_ht']/3)}, index = ['Force due to LL surcharge'])
display(A6)


6.34704


,Force(KN),Lever arm(m),Moment(KNm)
Force due to LL surcharge,357.211411,2.333333,833.493293


Seismic loads as per clause 218 of IRC 6

In [16]:
"""Inputs"""
zf = zone_factor(5) 
d = 1000 #Seismic importance factor as per table 19 of IRC 6
i = 1.2
f = 250 # Average response acceleration coefficient (see clause 218.5.1)
rrf = 3.0 #Response reduction factor as per table 20 of IRC 6 
ahh = ah(zf,sag(vib_period(d,f),'I'),rrf,1.2)
av = ahh*2/3
lam = max(math.degrees(math.atan(ahh/(1+av))),math.degrees(math.atan(ahh/(1-av))))
print(lam)

11.560130794217766


Due to self weight and superstructure loads

In [17]:
H_force = []
for index, i in enumerate(loads):
    H_force.append(round(loads[index]*ahh,5))
ht = [cg('rectangle', [abut['stem_width'],abut['stem_ht']], [0,0])[1], 
      cg('triangle_2', [abut['be_taperwidth'],abut['stem_ht']], [0,0])[1],
      cg('rectangle', [abut['cap_width'],abut['cap']], [0,abut['stem_ht']])[1],
      cg('rectangle', [abut['bw_width'],abut['backw_ht']], [0,abut['stem_ht']+abut['cap']])[1],
      cg('triangle_1', [abut['haunch']['stem_haunch'][0],-abut['haunch']['stem_haunch'][1]], [0,abut['stem_ht']])[1],
      cg('rectangle', [abut['haunch']['approach_slab_haunch_rec'][0],-abut['haunch']['approach_slab_haunch_rec'][1]], [0,abut['total_ht']-abut['approach_slab_dep']])[1],
      cg('triangle_1', [abut['haunch']['approach_slab_haunch_tri'][0],-abut['haunch']['approach_slab_haunch_tri'][1]], [0,abut['total_ht']-abut['approach_slab_dep']-abut['haunch']['approach_slab_haunch_rec'][1]])[1]]
mmt = []
for index, i in enumerate(H_force):
    mmt.append(round(H_force[index]*ht[index],5))
A7 = pd.DataFrame({"Weight":loads, 'Hz force(KN)':H_force,"Lever arm":ht,"Moment":mmt},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
# display(A7)
A7.loc[len(A7.index)] = [None, A7['Hz force(KN)'].sum(),None,A7['Moment'].sum()]
A7.loc[len(A7.index)] = [dl, dl*ahh, abut['total_ht']-box['pillar_hei']/3-abut['wc'],(dl*ahh)*(abut['total_ht']-box['pillar_hei']/3-abut['wc'])]
display(A7.rename(index={7:'Total',8:'Superstructure DL'}))

,Weight,Hz force(KN),Lever arm,Moment
Stem(rectangle),767.8125,138.20625,1.462500,202.12664
Stem(triangle),30.7125,5.52825,0.975000,5.39004
Cap,157.5000,28.35000,3.175000,90.01125
Backwall,225.2250,40.54050,5.212500,211.31736
Stem haunch,5.2500,0.94500,2.758330,2.60662
Approach slab haunch(rec),13.1250,2.36250,6.575000,15.53344
Approach slab haunch(tri),6.5625,1.18125,6.283330,7.42218
Total,NaN,217.11375,NaN,534.40753
Superstructure DL,3000.0000,540.00000,5.858333,3163.50000


Due to Earth pressure

In [18]:
C = ca(alpha, beta, delta, phi, lam, av)
k = ka(alpha, beta, delta, phi)
ep = (C-k)*h*unitwt_bfill 
A8 = pd.DataFrame({"Force":.5*ep*h*abut['width_of_abut'],"Lever arm":h/3,"Moment":(.5*ep*h*abut['width_of_abut'])*h/3},index = ['Seismic force due to backfill'])
display(A8)
print(ep)

,Force,Lever arm,Moment
Seismic force due to backfill,754.755322,2.233333,1685.620218


26.821440000000003


Dynamic Surcharge 

In [19]:
A9 = pd.DataFrame({"Force":(unitwt_bfill*(C-k)*hl*abut['width_of_abut']*h),"Lever arm":(0.66*abut['total_ht']),"Moment":((unitwt_bfill*(C-k)*hl*abut['width_of_abut']*h)*(0.66*abut['total_ht']))},index = ['Dynamic Surcharge'])
display(A9)
print(hl)
print(C-k)

,Force,Lever arm,Moment
Dynamic Surcharge,270.360115,4.62,1249.063732


1.2
0.20016


Summary of loads

In [20]:
#inputs
#Unfactored Forces and their moments

force_ver = [A1.loc[7,'Load'],A2.iloc[0,0],A2.iloc[1,0],A2.iloc[2,0],0,0,A3.iloc[2,0],0,0,0,0,0,0,0]
force_hz = [0,0,0,0,A3.iloc[0,0],A3.iloc[1,0],0,A6.iloc[0,0],A4.iloc[0,0],A5.iloc[0,0],A7.iloc[0,0],A7.iloc[7,1],A8.iloc[0,0],A9.iloc[0,0]]
moment_unf = [A1.iloc[7,2],A2.iloc[0,2],A2.iloc[1,2],A2.iloc[2,2],A3.iloc[0,2],A3.iloc[1,2],A3.iloc[2,2],A6.iloc[0,2],A4.iloc[0,2],A5.iloc[0,2],A7.iloc[8,3],A7.iloc[7,3],A8.iloc[0,2],A9.iloc[0,2]]

#Safety Factors 

non_ses = [1.35,1.35,1.5,1.75,1.5,0,1,1.2,1.5,0.9,0,0,0,0]
ses = [1.35,1.35,0,1.75,1,0,0,0.2,0.2,0.5,1.5,1.5,1.5,1]

In [21]:
#Factored responses (Non Seismic)

ns_force_ver = []
ns_force_hz = []
ns_moment = []
for index, i in enumerate(force_ver):
    ns_force_ver.append(round(force_ver[index]*non_ses[index],5))
for index, i in enumerate(force_hz):
    ns_force_hz.append(round(force_hz[index]*non_ses[index],5))
for index, i in enumerate(moment_unf):
    ns_moment.append(round(moment_unf[index]*non_ses[index],5))
A10 = pd.DataFrame({"Vertical Forces":ns_force_ver, 'Horizontal forces':ns_force_hz,"Moment":ns_moment},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
A10.loc[len(A10.index)] = [A10['Vertical Forces'].sum(), A10['Horizontal forces'].sum(),A10['Moment'].sum()]
display(A10.rename(index={14:'Total'}))

#Factored responses (Seismic)

ses_force_ver = []
ses_force_hz = []
ses_moment = []
for index, i in enumerate(force_ver):
    ses_force_ver.append(round(force_ver[index]*ses[index],5))
for index, i in enumerate(force_hz):
    ses_force_hz.append(round(force_hz[index]*ses[index],5))
for index, i in enumerate(moment_unf):
    ses_moment.append(round(moment_unf[index]*ses[index],5))
A11 = pd.DataFrame({"Vertical Forces":ses_force_ver, 'Horizontal forces':ses_force_hz,"Moment":ses_moment},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
A11.loc[len(A11.index)] = [A11['Vertical Forces'].sum(), A11['Horizontal forces'].sum(),A11['Moment'].sum()]
# display(A11.rename(index={14:'Total'}))


,Vertical Forces,Horizontal forces,Moment
Abutment self,1628.35312,0.00000,-305.82582
SS DL,4050.00000,0.00000,101.25000
SS Live load,1800.00000,0.00000,45.00000
Surface DL,437.50000,0.00000,10.93750
Earth Hz (dry),0.00000,1386.90273,3902.74430
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,373.56339,0.00000,-252.15528
Live load Surcharge,0.00000,428.65369,1000.19195
Braking,0.00000,332.40000,2725.68000
Temperature,0.00000,288.17875,1073.46585


**Foundation Stability**

Load from self weight of abutment

In [22]:
la_fromtoe = [cg('rectangle',[abut['stem_width'],abut['stem_ht']],[fdn['toe']+abut['be_taperwidth'],0])[0],
             cg('triangle_2',[abut['be_taperwidth'],abut['stem_ht']],[fdn['toe'],0])[0],
             cg('rectangle',[abut['cap_width'],abut['cap']],[fdn['toe']+abut['be_taperwidth'],abut['stem_ht']])[0],
             cg('rectangle',[abut['bw_width'],abut['backw_ht']],[fdn['toe']+abut['ttl_stembase_width']+abut['haunch']['stem_haunch'][0]-abut['bw_width'],abut['stem_ht']+abut['cap']])[0],
             cg('triangle_1',[abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']-abut['haunch']['stem_haunch'][1]])[0],
             cg('rectangle',[abut['haunch']['approach_slab_haunch_rec'][0],abut['haunch']['approach_slab_haunch_rec'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['stem_ht']-abut['haunch']['approach_slab_haunch_rec'][1]])[0],
             cg('triangle_1',[abut['haunch']['approach_slab_haunch_tri'][0],abut['haunch']['approach_slab_haunch_tri'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['stem_ht']-abut['haunch']['approach_slab_haunch_rec'][1]])[0]]
mom_abttoe = []
for index,i in enumerate(loads):
    mom_abttoe.append(loads[index]*la_fromtoe[index])
B1 = pd.DataFrame({"Load":loads,"Lever arm":la_fromtoe,"Moment":mom_abttoe},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
B1.loc[len(B1.index)] = [B1['Load'].sum(),None,B1['Moment'].sum()]
B1 = B1.rename(index={7:'Total'})
B1.loc[len(B1.index)] = [(unitwt_conc*fdn['len']*fdn['th']*fdn['wid']),fdn['len']/2,(unitwt_conc*fdn['len']*fdn['th']*fdn['wid'])*fdn['len']/2]
display(B1.rename(index={8:'Foundation Pad'}))


,Load,Lever arm,Moment
Stem(rectangle),767.8125,3.22500,2476.195312
Stem(triangle),30.7125,2.53333,77.804898
Cap,157.5000,3.35000,527.625000
Backwall,225.2250,3.90000,878.377500
Stem haunch,5.2500,3.98333,20.912482
Approach slab haunch(rec),13.1250,3.97500,52.171875
Approach slab haunch(tri),6.5625,4.01667,26.359397
Total,1206.1875,NaN,4059.446465
Foundation Pad,2649.3750,3.92500,10398.796875


Load from superstructure

In [23]:
arm_toe = [(fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c']),
          (fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c']),
          (fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c'])]
momentabt_toe = []
for index,i in enumerate(load):
    momentabt_toe.append(load[index]*arm_toe[index])
B2 = pd.DataFrame({"Load":load,"Lever arm":arm_toe,"Moment":momentabt_toe},index = ['Dead load','Live load','Surface Load'])
display(B2)

,Load,Lever arm,Moment
Dead load,3000,3.15,9450.0
Live load,1200,3.15,3780.0
Surface Load,250,3.15,787.5


Earth Pressure

In [24]:
ht1 = h + fdn['th']
ep_new = unitwt_bfill*ht1*ka(alpha, beta, delta, phi) #Earth pressure in KN/m^2.
force_new = 0.5*ep_new*(h+fdn['th'])*fdn['wid'] #Force due to earth pressure in KN.
forc_new = [force_new*math.cos(math.radians(delta)),force_new*math.cos(math.radians(delta)),force_new*math.sin(math.radians(delta))] #Horizontal and vertical components.
larm_abttoe = [-0.42*ht1, -0.33*ht1, (fdn['len']-fdn['heel']) ] #-ve for overturning moments.
print(force_new)
momen_abt_toe = []
for index,i in enumerate(forc_new):
    momen_abt_toe.append(forc_new[index]*larm_abttoe[index])
B3 = pd.DataFrame({'Force(KN)':forc_new, 'Lever arm(m)':larm_abttoe,'Moment(KNm)':momen_abt_toe}, index = ['Earth pressure Hz(dry)','Earth pressure Hz(wet)','Earth pressure Ver'])
display(B3)

1600.4061359999996


,Force(KN),Lever arm(m),Moment(KNm)
Earth pressure Hz(dry),1483.870730,-3.444,-5110.450794
Earth pressure Hz(wet),1483.870730,-2.706,-4015.354196
Earth pressure Ver,599.522691,3.850,2308.162359


Braking

In [25]:
br_forces = [max(F),-1*(max(F)*pt/box['span'])]
la_toe = [-(ht1+abut['approach_slab_dep']+pt),(fdn['len']-fdn['heel']+abut['haunch']['stem_haunch'][0]-abut['bw_width']-abut['bw_to_bearing_c'])]
m_toe = []
for index,i in enumerate(br_forces):
    m_toe.append(br_forces[index]*la_toe[index])
B4 = pd.DataFrame({'Force(KN)':br_forces, 'Lever arm(m)':la_toe,'Moment(KNm)':m_toe}, index = ['Braking load','Braking vertical reaction'])
display(B4)

,Force(KN),Lever arm(m),Moment(KNm)
Braking load,221.6000,-9.70,-2149.52000
Braking vertical reaction,-5.3184,3.15,-16.75296


Temperature Forces

In [26]:
La_toe = -(abut['stem_ht']+abut['cap']+bearing_ht+fdn['th'])
B5 = pd.DataFrame({'Force(KN)':frc, 'Lever arm(m)':La_toe,'Moment(KNm)': (frc*La_toe)}, index = ['Temperature force'])
display(B5)

,Force(KN),Lever arm(m),Moment(KNm)
Temperature force,320.198612,-5.225,-1673.037746


Live Load Surcharge

In [27]:
B6 = pd.DataFrame({'Force(KN)':(Pr*abut['width_of_abut']*abut['total_ht']), 'Lever arm(m)':-(abut['total_ht']+fdn['th'])/2,'Moment(KNm)':-((Pr*abut['width_of_abut']*abut['total_ht'])*(abut['total_ht']+fdn['th'])/2)}, index = ['Force due to LL surcharge'])
B6.loc[len(B6.index)] = [(unitwt_bfill*pt*abut['width_of_abut']*fdn['heel']), (fdn['heel']/2+fdn['toe']+abut['ttl_stembase_width']),(unitwt_bfill*pt*abut['width_of_abut']*fdn['heel'])*(fdn['heel']/2+fdn['toe']+abut['ttl_stembase_width'])]
display(B6.rename(index={1:'Vertical Surcharge'}))

,Force(KN),Lever arm(m),Moment(KNm)
Force due to LL surcharge,373.205952,-4.25,-1586.125296
Vertical Surcharge,806.400000,5.85,4717.440000


Backfill

In [28]:
fbfill = unitwt_bfill*fdn['heel']*fdn['wid']*h
la_filltoe = fdn['len']-fdn['heel']/2
B7 = pd.DataFrame({'Force(KN)':fbfill, 'Lever arm(m)':la_filltoe,'Moment(KNm)':fbfill*la_filltoe}, index = ['Backfill load'])
display(B7)

,Force(KN),Lever arm(m),Moment(KNm)
Backfill load,4824.0,5.85,28220.4


Seismic Forces

In [29]:
la_abutses = [-cg('rectangle',[abut['stem_width'],abut['stem_ht']],[fdn['toe']+abut['be_taperwidth'],fdn['th']])[1],
             -cg('triangle_2',[abut['be_taperwidth'],abut['stem_ht']],[fdn['toe'],fdn['th']])[1],
             -cg('rectangle',[abut['cap_width'],abut['cap']],[fdn['toe']+abut['be_taperwidth'],abut['stem_ht']+fdn['th']])[1],
             -cg('rectangle',[abut['bw_width'],abut['backw_ht']],[fdn['toe']+abut['ttl_stembase_width']+abut['haunch']['stem_haunch'][0],abut['stem_ht']+abut['cap']+fdn['th']])[1],
             -cg('triangle_1',[abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][1]],[fdn['toe']+abut['ttl_stembase_width'],abut['stem_ht']-abut['haunch']['stem_haunch'][1]+fdn['th']])[1],
             -cg('rectangle',[abut['haunch']['approach_slab_haunch_rec'][0],abut['haunch']['approach_slab_haunch_rec'][1]],[fdn['toe']+abut['cap_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['haunch']['approach_slab_haunch_rec'][1]+fdn['th']])[1],
             -cg('triangle_1',[abut['haunch']['approach_slab_haunch_tri'][0],abut['haunch']['approach_slab_haunch_tri'][1]],[fdn['toe']+abut['cap_width'],abut['stem_ht']+abut['cap']+abut['backw_ht']-abut['haunch']['approach_slab_haunch_rec'][1]+fdn['th']])[1]]
mmt_toeses = []
for index, i in enumerate(H_force):
    mmt_toeses.append(round(H_force[index]*la_abutses[index],5))
B8 = pd.DataFrame({"Weight":loads, 'Hz force(KN)':H_force,"Lever arm":la_abutses,"Moment":mmt_toeses},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
# display(A7)
B8.loc[len(B8.index)] = [None, B8['Hz force(KN)'].sum(),None,B8['Moment'].sum()]
B8 = B8.rename(index={7:'Total'})
B8.loc[len(B8.index)] = [B1.iloc[8,0],B1.iloc[8,0]*ahh,-fdn['th']/2,-B1.iloc[8,0]*ahh*fdn['th']/2]
B8 = B8.rename(index={8:'Foundation Pad'})
display(B8)

,Weight,Hz force(KN),Lever arm,Moment
Stem(rectangle),767.8125,138.20625,-2.96250,-409.436020
Stem(triangle),30.7125,5.52825,-2.47500,-13.682420
Cap,157.5000,28.35000,-4.67500,-132.536250
Backwall,225.2250,40.54050,-6.71250,-272.128110
Stem haunch,5.2500,0.94500,-4.34167,-4.102880
Approach slab haunch(rec),13.1250,2.36250,-8.37500,-19.785940
Approach slab haunch(tri),6.5625,1.18125,-8.41667,-9.942190
Total,NaN,217.11375,NaN,-861.613810
Foundation Pad,2649.3750,476.88750,-0.75000,-357.665625


Superstructure DL

In [30]:
B9 = pd.DataFrame({"Weight":dl, "Hz Force(KN)":dl*ahh, "Lever arm":-(abut['total_ht']-box['pillar_hei']/3-abut['wc']+fdn['th']),"Moment":-(dl*ahh)*(abut['total_ht']-box['pillar_hei']/3-abut['wc']+fdn['th'])}, index = ['Superstructure DL'])
display(B9)

,Weight,Hz Force(KN),Lever arm,Moment
Superstructure DL,3000,540.0,-7.358333,-3973.5


Earth Pressure

In [31]:
ep_fdn = (C-k)*ht1*unitwt_bfill
forc_ep = .5*ep_fdn*ht1*fdn['wid']
la_eptoe = -0.66*ht1
B10 = pd.DataFrame({"Force(KN)":forc_ep, "Lever arm":la_eptoe,"Moment":forc_ep*la_eptoe}, index = ['Seismic force due to backfill'])
display(B10)

,Force(KN),Lever arm,Moment
Seismic force due to backfill,1211.288256,-5.412,-6555.492041


Dynamic Surcharge

In [32]:
B11 = pd.DataFrame({"Force":(unitwt_bfill*(C-k)*hl*fdn['wid']*abut['total_ht']),"Lever arm":la_eptoe,"Moment":((unitwt_bfill*(C-k)*hl*fdn['wid']*abut['total_ht'])*la_eptoe)},index = ['Dynamic Surcharge'])
display(B11)

,Force,Lever arm,Moment
Dynamic Surcharge,302.64192,-5.412,-1637.898071


In [33]:
#inputs
#Unfactored Forces and their moments

ver_force = [B1.iloc[7,0],B2.iloc[0,0],B2.iloc[1,0],B2.iloc[2,0],B1.iloc[8,0],0,0,B3.iloc[2,0],0,B6.iloc[1,0],0,0,B7.iloc[0,0],B4.iloc[1,0],0,0,0,0,0]
h_forc = [0,0,0,0,0,0,B3.iloc[1,0],0,B6.iloc[0,0],0,B4.iloc[0,0],B5.iloc[0,0],0,0,B8.iloc[7,1],B8.iloc[8,1],B9.iloc[0,1],B10.iloc[0,0],B11.iloc[0,0]]
unf_moment_res = [B1.iloc[7,2],B2.iloc[0,2],B2.iloc[1,2],B2.iloc[2,2],B1.iloc[8,2],0,0,B3.iloc[2,2],0,B6.iloc[1,2],0,0,B7.iloc[0,2],0,0,0,0,0,0]
unf_moment_over = [0,0,0,0,0,0,B3.iloc[1,2],0,B6.iloc[0,2],0,B4.iloc[0,2],B5.iloc[0,2],0,B4.iloc[1,2],B8.iloc[7,3],B8.iloc[8,3],B9.iloc[0,3],B10.iloc[0,2],B11.iloc[0,2]]
fr_coeff = 0.65
print(ver_force)
##Safety Factors (Basic)

bas_R = [0.9,0.9,0,1,0.9,1,0,1,0,0,0,0,0.9,0,0,0,0,0,0]
bas_O = [1.1,1.1,1.5,1.35,1.1,1.5,0,1.5,1.2,1.2,1.15,1.5,1.1,1.15,0,0,0,0,0]

##Safety Factors (Seismic)

ses_R = [0.9,0.9,0,1,0.9,0,1,1,0,0,0,0,0.9,0,0,0,0,0,0]
ses_O = [1.1,1.1,0,1.35,1.1,0,1,1,0,0,0.2,0,1.1,0.2,1.5,1.5,1.5,1.5,1.5]

[1206.1875, 3000, 1200, 250, 2649.375, 0, 0, 599.5226906888827, 0, 806.4000000000001, 0, 0, 4824.0, -5.3184000000000005, 0, 0, 0, 0, 0]


In [34]:
#Factored Responses (Basic)
bas_force_ver = []
bas_force_hz = []
bas_momentR = []
bas_momentO = []
for index, i in enumerate(ver_force):
     bas_force_ver.append(round(ver_force[index]*bas_R[index],5))
     bas_momentR.append(round(unf_moment_res[index]*bas_R[index],5))
bas_force_ver[13] = ver_force[13]*bas_O[13]
for index, i in enumerate(h_forc):
    bas_force_hz.append(round(h_forc[index]*bas_O[index],5))
    bas_momentO.append(round(unf_moment_over[index]*bas_O[index],5))
B12 = pd.DataFrame({"Vertical Forces":bas_force_ver, 'Horizontal forces':bas_force_hz,'Resisting Moment':bas_momentR,'Overturning moment':bas_momentO},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(HZ)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
B12.loc[len(B12.index)] = [B12['Vertical Forces'].sum(), B12['Horizontal forces'].sum(),B12['Resisting Moment'].sum(),B12['Overturning moment'].sum()]
display(B12.rename(index={19:'Total'}))

#Factored Responses (Seismic)
ses_force_ver = []
ses_force_hz = []
ses_momentR = []
ses_momentO = []
for index, i in enumerate(ver_force):
    ses_force_ver.append(round(ver_force[index]*ses_R[index],5))
    ses_momentR.append(round(unf_moment_res[index]*ses_R[index],5))
ses_force_ver[13] = ver_force[13]*ses_O[13]
for index, i in enumerate(h_forc):
    ses_force_hz.append(round(h_forc[index]*ses_O[index],5))
    ses_momentO.append(round(unf_moment_over[index]*ses_O[index],5))
B13 = pd.DataFrame({"Vertical Forces":ses_force_ver, 'Horizontal forces':ses_force_hz,'Resisting Moment':ses_momentR,'Overturning moment':ses_momentO},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(HZ)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
B13.loc[len(B13.index)] = [B13['Vertical Forces'].sum(), B13['Horizontal forces'].sum(),B13['Resisting Moment'].sum(),B13['Overturning moment'].sum()]
# display(B13.rename(index={19:'Total'}))

#Overturning and sliding check
fact_basic = [B12.iloc[19,2]/-(B12.iloc[19,3]), (fr_coeff*B12.iloc[19,0])/B12.iloc[19,1]]
fact_ses = [B13.iloc[19,2]/-(B13.iloc[19,3]), (fr_coeff*B13.iloc[19,0])/B13.iloc[19,1]]
F = pd.DataFrame({"Basic":fact_basic, "Seismic":fact_ses}, index = ['Overturning','Sliding'])
# display(F)

,Vertical Forces,Horizontal forces,Resisting Moment,Overturning moment
Abutment self,1085.56875,0.00000,3653.50182,0.00000
SS DL,2700.00000,0.00000,8505.00000,0.00000
SS Live load,0.00000,0.00000,0.00000,0.00000
Surface DL,250.00000,0.00000,787.50000,0.00000
Foundation,2384.43750,0.00000,9358.91719,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,0.00000,0.00000,-0.00000
Earth ver,599.52269,0.00000,2308.16236,0.00000
Live load Surcharge(HZ),0.00000,447.84714,0.00000,-1903.35036
Live load surcharge(Ver),0.00000,0.00000,0.00000,0.00000


Bearing Pressure foundation

In [35]:
#Inputs
ex = -fdn['len']/2
ey = 0
c1 = [1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0]
c2 = [1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1]
c3 = [1,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0]

In [36]:
print(len(unf_moment_over))
tot_unf_M = []
for i,j in zip(unf_moment_res,unf_moment_over):
    if i != 0:
        tot_unf_M.append(i)
    elif j!=0:
        tot_unf_M.append(j)
tot_unf_M.insert(5,0)
# print(tot_unf_M)

C1 = pd.DataFrame({"Vertical Forces":ver_force, 'Horizontal forces':h_forc,'Moment':tot_unf_M},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C1.loc[len(C1.index)] = [C1['Vertical Forces'].sum(), C1['Horizontal forces'].sum(),C1['Moment'].sum()]
display(C1.rename(index={19:'Total'}))



19


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.187500,0.000000,4059.446465
SS DL,3000.000000,0.000000,9450.000000
SS Live load,1200.000000,0.000000,3780.000000
Surface DL,250.000000,0.000000,787.500000
Foundation,2649.375000,0.000000,10398.796875
Earth Hz (dry),0.000000,0.000000,0.000000
Earth Hz (wet),0.000000,1483.870730,-4015.354196
Earth ver,599.522691,0.000000,2308.162359
Live load surcharge (Hz),0.000000,373.205952,-1586.125296
Live load surcharge(Ver),806.400000,0.000000,4717.440000


In [37]:
new_M = []
for i,j,k in zip(tot_unf_M,ver_force,h_forc):
    new_M.append(i+j*ex+k*ey)
print(new_M)
C2 = pd.DataFrame({"Vertical Forces":ver_force, 'Horizontal forces':h_forc,'Moment':new_M},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
display(C2)


[-674.8394729999995, -2325.0, -930.0, -193.75, 0.0, 0.0, -4015.3541955121013, -44.964201801666604, -1586.1252960000002, 1552.3200000000002, -2149.52, -1673.0377464705884, 9286.199999999997, 4.121759999999998, -861.61381, -357.665625, -3973.5, -6555.492041471999, -1637.8980710399999]


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.187500,0.000000,-674.839473
SS DL,3000.000000,0.000000,-2325.000000
SS Live load,1200.000000,0.000000,-930.000000
Surface DL,250.000000,0.000000,-193.750000
Foundation,2649.375000,0.000000,0.000000
Earth Hz (dry),0.000000,0.000000,0.000000
Earth Hz (wet),0.000000,1483.870730,-4015.354196
Earth ver,599.522691,0.000000,-44.964202
Live load surcharge (Hz),0.000000,373.205952,-1586.125296
Live load surcharge(Ver),806.400000,0.000000,1552.320000


In [38]:
# combination 1
c1_verf = []
c1_hzf = []
c1_mom = []
for index, i in enumerate(c1):
    c1_verf.append(round(c1[index]*ver_force[index],5))
    c1_hzf.append(round(c1[index]*h_forc[index],5))
    c1_mom.append(round(c1[index]*new_M[index],5))
C3 = pd.DataFrame({"Vertical Forces":c1_verf, 'Horizontal forces':c1_hzf,'Moment':c1_mom},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C3.loc[len(C3.index)] = [C3['Vertical Forces'].sum(), C3['Horizontal forces'].sum(),C3['Moment'].sum()]
display(C3.rename(index={19:'Total'}))

#Combination 2
c2_verf = []
c2_hzf = []
c2_mom = []
for index, i in enumerate(c1):
    c2_verf.append(round(c2[index]*ver_force[index],5))
    c2_hzf.append(round(c2[index]*h_forc[index],5))
    c2_mom.append(round(c2[index]*new_M[index],5))
C4 = pd.DataFrame({"Vertical Forces":c2_verf, 'Horizontal forces':c2_hzf,'Moment':c2_mom},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C4.loc[len(C4.index)] = [C4['Vertical Forces'].sum(), C4['Horizontal forces'].sum(),C4['Moment'].sum()]
display(C4.rename(index={19:'Total'}))

#Combination 3
c3_verf = []
c3_hzf = []
c3_mom = []
for index, i in enumerate(c1):
    c3_verf.append(round(c3[index]*ver_force[index],5))
    c3_hzf.append(round(c3[index]*h_forc[index],5))
    c3_mom.append(round(c3[index]*new_M[index],5))
C5 = pd.DataFrame({"Vertical Forces":c3_verf, 'Horizontal forces':c3_hzf,'Moment':c3_mom},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load surcharge (Hz)','Live load surcharge(Ver)','Braking','Temperature','Backfill','Braking uplift','SS DL(seismic)','Seismic Foundation','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
C5.loc[len(C5.index)] = [C5['Vertical Forces'].sum(), C5['Horizontal forces'].sum(),C5['Moment'].sum()]
display(C5.rename(index={19:'Total'}))


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.18750,0.00000,-674.83947
SS DL,3000.00000,0.00000,-2325.00000
SS Live load,1200.00000,0.00000,-930.00000
Surface DL,250.00000,0.00000,-193.75000
Foundation,2649.37500,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,0.00000,-0.00000
Earth ver,599.52269,0.00000,-44.96420
Live load surcharge (Hz),0.00000,373.20595,-1586.12530
Live load surcharge(Ver),806.40000,0.00000,1552.32000


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.18750,0.00000,-674.83947
SS DL,3000.00000,0.00000,-2325.00000
SS Live load,1200.00000,0.00000,-930.00000
Surface DL,250.00000,0.00000,-193.75000
Foundation,2649.37500,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,1483.87073,-4015.35420
Earth ver,599.52269,0.00000,-44.96420
Live load surcharge (Hz),0.00000,373.20595,-1586.12530
Live load surcharge(Ver),806.40000,0.00000,1552.32000


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.18750,0.00000,-674.83947
SS DL,3000.00000,0.00000,-2325.00000
SS Live load,0.00000,0.00000,-0.00000
Surface DL,250.00000,0.00000,-193.75000
Foundation,2649.37500,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,1483.87073,-4015.35420
Earth ver,599.52269,0.00000,-44.96420
Live load surcharge (Hz),0.00000,0.00000,-0.00000
Live load surcharge(Ver),0.00000,0.00000,0.00000


***Design of abutment stem***

In [39]:
axf_c = 0.446*fck*abut['ttl_stembase_width']*abut['width_of_abut']*0.1*1000
res = [A10.iloc[14,0],A11.iloc[14,0]]
des_mom = [A10.iloc[14,2],A11.iloc[14,2]]
if axf_c > max(res):
    print("The member is designed for pure flexure.")
print(axf_c)

The member is designed for pure flexure.
15172.920000000004


Calculation of limiting values

In [40]:
#Inputs

Ecu = 0.0035 #Limiting strain in concrete.
Es = 0.87*fy/E
ce = 75 #Effective cover in mm.
d = (abut['ttl_stembase_width']*1000) - ce - 10 #Effective depth of beam.
beta1 = 0.80952
beta2 = 0.416
fyd = fy/1.15
fctm = 0.259*pow(fck,2/3)
dia = 20 #Diameter of reinforcement bars in mm.
print(d)

1265.0


In [41]:
xu_lim = (Ecu*d)/(Ecu+Es) #Limiting depth of neutral axis in mm
cg_steel = d-0.416*xu_lim #CG from level of steel in mm.
comp_force = (0.446*fck*beta1*(1000*abut['width_of_abut'])*xu_lim)/1000 
mu_lim = (comp_force*cg_steel)/1000
print(mu_lim)

66755.93320192143


Calculation of Moment of resistance of section

In [42]:
xu = int(input("Enter a trial value of depth of NA:"))
cgs = d-0.416*xu
compf = (0.446*fck*beta1*(1000*abut['width_of_abut'])*xu)/1000 
mor = (cgs*compf)/1000
x = na(xu,cgs,compf,max(des_mom))
# print(x)

Enter a trial value of depth of NA:150


Calculation of steel required

In [54]:
steel_req = (compf*1000)/fyd #Area of reinforcements required in mm^2.

#Minimum steel required
ast_minr = round(0.26*fctm/fy,5) if 0.26*fctm/fy>0.0013 else 0.0013
ast_min = ast_minr*abut['width_of_abut']*d
Check1 = "OK" if steel_req>=ast_min else "Not OK"
if Check1 == "Not OK":
    x = na(xu,cgs,compf,max(des_mom))

#Maximum steel required
ast_max = 0.025*(abut['width_of_abut']*abut['ttl_stembase_width'])*pow(10,6)
Check2 = "OK" if steel_req<=ast_max else "Not OK"
if Check2 == "Not OK":
    x = na(xu,cgs,compf,max(des_mom))
print(Check2)
print(steel_req)

OK
31389.332284800006


In [44]:
area_bar = math.pi * pow(dia,2)/4
spacing = 120
ast_pro = area_bar*abut['width_of_abut']*1000/spacing
while ast_pro<=steel_req:
    spacing = int(input("Enter lower value of spacing:"))
    ast_pro = area_bar*abut['width_of_abut']*1000/spacing
print(spacing)
print("OK")

Enter lower value of spacing:70
70
OK


Shear Check

In [45]:
v_min = min(res)*1000
ac = abut['width_of_abut']*1000*d
sigma_c = v_min/ac
k = 1+ pow(200/d,.5) if (1+ pow(200/d,.5))<=2 else 2
row_1 = ast_pro/(abut['width_of_abut']*1000*d)
print(row_1)
t_min = 0.031*pow(k,1.5)*pow(fck,0.5)
vrdc_min = (t_min + 0.15*sigma_c)*abut['width_of_abut']*1000*d/1000
if (0.12*k*pow(80*row_1*fck,0.33)+0.15*sigma_c)*abut['width_of_abut']*1000*d/1000>=vrdc_min:
    vrdc = (0.12*k*pow(80*row_1*fck,0.33)+0.15*sigma_c)*abut['width_of_abut']*1000*d/1000
else:
    vrdc = vrdc_min
    
#Check
shear_f = max([A10.iloc[14,1],A11.iloc[14,1]])
print(vrdc,shear_f)
if vrdc >= shear_f:
    print("OK")
else:
    print("Not OK")

0.0035478177906152378
4530.624015687442 4080.34589
OK


Calculation of Crack Width

In [46]:
#Inputs
hc_eff = min([2.5*(abut['ttl_stembase_width']-d), (abut['ttl_stembase_width']-xu)/3, abut['ttl_stembase_width']/2])
ac_eff = abut['width_of_abut']*hc_eff
k_t = 0.5
f_cm = fck+10
c = 75
#Factors
rare = [1,1,1,1.2,1,0,1,0.8,0.75,0.6,0,0,0,0]
freq = [1,1,0.75,1.2,1,0,1,0,0.2,0.5,0,0,0,0]
quasi = [1,1,0,1.2,1,0,1,0,0,0.5,0,0,0,0]

# Rare combination
v_rare = []
hz_rare = []
mom_rare = []
for index,i in enumerate(rare):
    v_rare.append(round(rare[index]*force_ver[index],5))
    hz_rare.append(round(rare[index]*force_hz[index],5))
    mom_rare.append(round(rare[index]*moment_unf[index],5))
W1 = pd.DataFrame({"Vertical Forces":v_rare, 'Horizontal forces':hz_rare,"Moment":mom_rare},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
W1.loc[len(W1.index)] = [W1['Vertical Forces'].sum(), W1['Horizontal forces'].sum(), W1['Moment'].sum()]
display(W1.rename(index={14:'Total'}))

# Frequent combination
v_freq = []
hz_freq = []
mom_freq = []
for index,i in enumerate(freq):
    v_freq.append(round(freq[index]*force_ver[index],5))
    hz_freq.append(round(freq[index]*force_hz[index],5))
    mom_freq.append(round(freq[index]*moment_unf[index],5))
W2 = pd.DataFrame({"Vertical Forces":v_freq, 'Horizontal forces':hz_freq,"Moment":mom_freq},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
W2.loc[len(W2.index)] = [W2['Vertical Forces'].sum(), W2['Horizontal forces'].sum(), W2['Moment'].sum()]
display(W2.rename(index={14:'Total'}))

# Quasi-permanent combination
v_quasi = []
hz_quasi = []
mom_quasi = []
for index,i in enumerate(quasi):
    v_quasi.append(round(quasi[index]*force_ver[index],5))
    hz_quasi.append(round(quasi[index]*force_hz[index],5))
    mom_quasi.append(round(quasi[index]*moment_unf[index],5))
W3 = pd.DataFrame({"Vertical Forces":v_quasi, 'Horizontal forces':hz_quasi,"Moment":mom_quasi},index = ['Abutment self','SS DL','SS Live load','Surface DL','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge','Braking','Temperature','SS DL(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
W3.loc[len(W3.index)] = [W3['Vertical Forces'].sum(), W3['Horizontal forces'].sum(), W3['Moment'].sum()]
display(W3.rename(index={14:'Total'}))

,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.18750,0.00000,-226.53764
SS DL,3000.00000,0.00000,75.00000
SS Live load,1200.00000,0.00000,30.00000
Surface DL,300.00000,0.00000,7.50000
Earth Hz (dry),0.00000,924.60182,2601.82953
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,373.56339,0.00000,-252.15528
Live load Surcharge,0.00000,285.76913,666.79463
Braking,0.00000,166.20000,1362.84000
Temperature,0.00000,192.11917,715.64390


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.18750,0.00000,-226.53764
SS DL,3000.00000,0.00000,75.00000
SS Live load,900.00000,0.00000,22.50000
Surface DL,300.00000,0.00000,7.50000
Earth Hz (dry),0.00000,924.60182,2601.82953
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,373.56339,0.00000,-252.15528
Live load Surcharge,0.00000,0.00000,0.00000
Braking,0.00000,44.32000,363.42400
Temperature,0.00000,160.09931,596.36991


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.18750,0.00000,-226.53764
SS DL,3000.00000,0.00000,75.00000
SS Live load,0.00000,0.00000,0.00000
Surface DL,300.00000,0.00000,7.50000
Earth Hz (dry),0.00000,924.60182,2601.82953
Earth Hz (wet),0.00000,0.00000,0.00000
Earth ver,373.56339,0.00000,-252.15528
Live load Surcharge,0.00000,0.00000,0.00000
Braking,0.00000,0.00000,0.00000
Temperature,0.00000,160.09931,596.36991


In [47]:
bm_crack = min([W1.iloc[14,2],W2.iloc[14,2],W3.iloc[14,2]])
stress_act = (bm_crack*pow(10,6))/(ast_pro*cgs)
fct_eff = 0.259*pow(fck,2/3)
e_cm = 22*pow(f_cm/12.5,0.3)*1000
ro1_eff = ast_pro/ac_eff
alp_e = E/e_cm
if (stress_act-(k_t*fct_eff*(1+alp_e*ro1_eff))/ro1_eff)/E >= 0.6*stress_act/E:
    diff_e = (stress_act-(k_t*fct_eff*(1+alp_e*ro1_eff))/ro1_eff)/E
else:
    diff_e = 0.6*stress_act/E
sr_max = 3.4*c+(0.17*dia)/ro1_eff

#Crack width in mm
cr_wid = sr_max*diff_e
#Check
print("OK") if cr_wid <= 0.3 else print("Not OK")

OK


Horizontal Reinforcements

In [48]:
check_ar = math.pi*(pow(dia,2)/4)*(1000/spacing)
print(check_ar)
ar_list = [0.25*check_ar, (0.001*1000*abut['ttl_stembase_width']*1000)/2]
spacing_h = int(input("Enter trial spacing:"))
dia_h = int(input("Enter a trial diameter:"))
while (math.pi*pow(dia_h,2)/4*1000/(spacing))<=max(ar_list):
    dia_h = int(input("Enter another trial value of diameter:"))
    spacing_h = int(input("Enter another trial value:"))
ast_hpro = math.pi*pow(dia_h,2)/4*(1000/(spacing_h))
print(ast_hpro)   

4487.9895051282765
Enter trial spacing:150
Enter a trial diameter:16
1340.4128655316451


Vertical Reinforcements

In [49]:
# check_ar = math.pi*(pow(dia,2)/4)*(1000/spacing)
ver_dia = 20 
ver_spc = 180
ast_vert = math.pi*pow(ver_dia,2)/4*(1000/(ver_spc))
print(ast_vert) 

1745.3292519943295


**Design of open foundation**

In [50]:
display(C2)

,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.187500,0.000000,-674.839473
SS DL,3000.000000,0.000000,-2325.000000
SS Live load,1200.000000,0.000000,-930.000000
Surface DL,250.000000,0.000000,-193.750000
Foundation,2649.375000,0.000000,0.000000
Earth Hz (dry),0.000000,0.000000,0.000000
Earth Hz (wet),0.000000,1483.870730,-4015.354196
Earth ver,599.522691,0.000000,-44.964202
Live load surcharge (Hz),0.000000,373.205952,-1586.125296
Live load surcharge(Ver),806.400000,0.000000,1552.320000


In [51]:
bas_1 = [1.35,1.35,1.5,1.75,1.35,1.5,1.5,1.5,1.2,1.2,1.15,.9,1.35,0,0,0,0,0,0]
bas_2 = [1,1,1.3,1,1,1.3,1.3,1.3,1,1,1,.8,1,0,0,0,0,0,0]
seis_com = [1.35,1.35,0.75,1.75,1.35,0,1,1,.2,.2,.2,.5,1.35,0,1.5,1.5,1.5,1.5,1.5]

#Basic combination 1
v_bas_1 = []
hz_bas_1 = []
mom_bas_1 = []
for index,i in enumerate(bas_1):
    v_bas_1.append(round(bas_1[index]*ver_force[index],5))
    hz_bas_1.append(round(bas_1[index]*h_forc[index],5))
    mom_bas_1.append(round(bas_1[index]*new_M[index],5))
X1 = pd.DataFrame({"Vertical Forces":v_bas_1, 'Horizontal forces':hz_bas_1,"Moment":mom_bas_1},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(hz)','Live load Surcharge(ver)','Braking','Temperature','backfill','Braking uplift','SS DL(seismic)','Foundation(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
X1.loc[len(X1.index)] = [X1['Vertical Forces'].sum(), X1['Horizontal forces'].sum(),X1['Moment'].sum()]
display(X1.rename(index={19:'Total'}))

#Basic combination 2
v_bas_2 = []
hz_bas_2 = []
mom_bas_2 = []
for index,i in enumerate(bas_2):
    v_bas_2.append(round(bas_2[index]*ver_force[index],5))
    hz_bas_2.append(round(bas_2[index]*h_forc[index],5))
    mom_bas_2.append(round(bas_2[index]*new_M[index],5))
X2 = pd.DataFrame({"Vertical Forces":v_bas_2, 'Horizontal forces':hz_bas_2,"Moment":mom_bas_2},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(hz)','Live load Surcharge(ver)','Braking','Temperature','backfill','Braking uplift','SS DL(seismic)','Foundation(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
X2.loc[len(X2.index)] = [X2['Vertical Forces'].sum(), X2['Horizontal forces'].sum(),X2['Moment'].sum()]
display(X2.rename(index={19:'Total'}))

#Seismic combination
v_seis = []
hz_seis = []
mom_seis = []
for index,i in enumerate(seis_com):
    v_seis.append(round(seis_com[index]*ver_force[index],5))
    hz_seis.append(round(seis_com[index]*h_forc[index],5))
    mom_seis.append(round(seis_com[index]*new_M[index],5))
X3 = pd.DataFrame({"Vertical Forces":v_seis, 'Horizontal forces':hz_seis,"Moment":mom_seis},index = ['Abutment self','SS DL','SS Live load','Surface DL','Foundation','Earth Hz (dry)','Earth Hz (wet)','Earth ver','Live load Surcharge(hz)','Live load Surcharge(ver)','Braking','Temperature','backfill','Braking uplift','SS DL(seismic)','Foundation(seismic)','Abutment DL(seismic)','Earth p (seismic)','Dyn surcharge(Seismic)'])
X3.loc[len(X3.index)] = [X3['Vertical Forces'].sum(), X3['Horizontal forces'].sum(),X3['Moment'].sum()]
display(X3.rename(index={19:'Total'}))


,Vertical Forces,Horizontal forces,Moment
Abutment self,1628.35312,0.00000,-911.03329
SS DL,4050.00000,0.00000,-3138.75000
SS Live load,1800.00000,0.00000,-1395.00000
Surface DL,437.50000,0.00000,-339.06250
Foundation,3576.65625,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,2225.80610,-6023.03129
Earth ver,899.28404,0.00000,-67.44630
Live load Surcharge(hz),0.00000,447.84714,-1903.35036
Live load Surcharge(ver),967.68000,0.00000,1862.78400


,Vertical Forces,Horizontal forces,Moment
Abutment self,1206.1875,0.00000,-674.83947
SS DL,3000.0000,0.00000,-2325.00000
SS Live load,1560.0000,0.00000,-1209.00000
Surface DL,250.0000,0.00000,-193.75000
Foundation,2649.3750,0.00000,0.00000
Earth Hz (dry),0.0000,0.00000,0.00000
Earth Hz (wet),0.0000,1929.03195,-5219.96045
Earth ver,779.3795,0.00000,-58.45346
Live load Surcharge(hz),0.0000,373.20595,-1586.12530
Live load Surcharge(ver),806.4000,0.00000,1552.32000


,Vertical Forces,Horizontal forces,Moment
Abutment self,1628.35312,0.00000,-911.03329
SS DL,4050.00000,0.00000,-3138.75000
SS Live load,900.00000,0.00000,-697.50000
Surface DL,437.50000,0.00000,-339.06250
Foundation,3576.65625,0.00000,0.00000
Earth Hz (dry),0.00000,0.00000,0.00000
Earth Hz (wet),0.00000,1483.87073,-4015.35420
Earth ver,599.52269,0.00000,-44.96420
Live load Surcharge(hz),0.00000,74.64119,-317.22506
Live load Surcharge(ver),161.28000,0.00000,310.46400


In [52]:
moi_fdn = moi('rectangle', [9,7.85])[0]
print(moi_fdn)

362.8025
